# Simulated Function Calling with Qwen Models (Featherless API)

This notebook demonstrates how to simulate function (tool) calling with Qwen models using the OpenAI Python SDK and the Featherless API.

Qwen models use a different tool call format than OpenAI models. See the [Qwen function call documentation](https://qwen.readthedocs.io/en/latest/framework/function_call.html) for details.

---

## Prerequisites
1. Sign up for an account at [Featherless](https://featherless.ai/register)
2. Subscribe to a plan and get your API key from [API Keys](https://featherless.ai/account/api-keys)

## Setup
First, let's import the required libraries and set up our API key.

In [1]:
# Install the OpenAI Python SDK if you haven't already
# !pip install openai

In [2]:
from openai import OpenAI

# Initialize the OpenAI client with your endpoint and API key
client = OpenAI(
    base_url="https://api.featherless.ai/v1",
    api_key="YOUR FEATHERLESS API KEY",  # Replace with your actual API key
)

## Qwen Tool Call Format

Qwen models expect tool calls in a special format, for example:
```
<|im_start|>tool_call
{"name": "get_weather", "arguments": {"location": "Paris, France"}}
<|im_end|>
```
We will instruct the model to always respond with this format.

In [3]:
def extract_tool_calls(response_text):
    """
    Extract all tool call JSON objects from the Qwen output.
    Handles both <|im_start|>tool_call ... <|im_end|> and plain JSON objects, including multiple per output.
    Returns a list of dicts.
    """
    import re, json

    tool_calls = []

    # First, extract all <|im_start|>tool_call ... <|im_end|> blocks
    matches = re.findall(r'<\|im_start\|>tool_call\s*(\{.*?\})\s*<\|im_end\|>', response_text, re.DOTALL)
    for match in matches:
        try:
            tool_calls.append(json.loads(match))
        except Exception:
            pass

    # If none found, or to also support plain JSON, extract all JSON objects from the text (one per line)
    if not tool_calls:
        for line in response_text.splitlines():
            line = line.strip()
            if line.startswith("{") and line.endswith("}"):
                try:
                    tool_calls.append(json.loads(line))
                except Exception:
                    pass

    return tool_calls if tool_calls else None

## Example 1: Simulate a Single Function Call (`get_weather`)

We instruct the model to always respond with a tool call for `get_weather`.

In [4]:
response = client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant.\n"
                "When asked a question, respond ONLY in the following format:\n"
                "<|im_start|>tool_call\n"
                '{"name": "get_weather", "arguments": {"location": "<city, country>"}}\n'
                "<|im_end|>\n"
                "Do not answer the question directly or add any explanations. Only output the tool_call block."
            ),
        },
        {
            "role": "user",
            "content": "What is the weather like in Paris today?"
        }
    ],
)
print("Raw Qwen Output:\n", response.choices[0].message.content)
tool_call = extract_tool_calls(response.choices[0].message.content)
print("\nExtracted Tool Call:", tool_call)

Raw Qwen Output:
 <think>
Okay, the user is asking about the weather in Paris today. I need to call the get_weather function. Let me check the format they want. They specified to respond only with the tool_call block. The function name is get_weather, and the arguments should include location as "<city, country>". So Paris is the city, and the country is France. I should make sure to format it correctly without any extra text or explanations. Just the JSON inside the tool_call tags. Let me double-check the syntax to avoid errors. Alright, that should do it.
</think>

tool_call
{"name": "get_weather", "arguments": {"location": "Paris, France"}}

Extracted Tool Call: [{'name': 'get_weather', 'arguments': {'location': 'Paris, France'}}]


## Example 2: Simulate Multiple Functions (`get_weather` or `get_time`)

Here, we tell the model it can call either `get_weather` or `get_time`, and to always respond with the correct tool call.

In [5]:
response = client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant.\n"
                "You have access to two functions:\n"
                '1. get_weather(location: str)\n'
                '2. get_time(location: str)\n'
                "When asked a question, respond ONLY in the following format:\n"
                "<|im_start|>tool_call\n"
                '{"name": "<function_name>", "arguments": {"location": "<city, country>"}}\n'
                "<|im_end|>\n"
                "Do not answer the question directly or add any explanations. Only output the tool_call block."
            ),
        },
        {
            "role": "user",
            "content": "What's the weather and time in Tokyo?"
        }
    ],
)
print("Raw Qwen Output:\n", response.choices[0].message.content)
tool_call = extract_tool_calls(response.choices[0].message.content)
print("\nExtracted Tool Call:", tool_call)

Raw Qwen Output:
 <think>
Okay, the user is asking for both the weather and the time in Tokyo. Let me check the functions available. There's get_weather and get_time, both requiring a location. Since the user mentioned Tokyo, I need to call each function once with Tokyo as the location. I should make sure to format the response correctly as two separate tool calls. No need to add any extra information, just the JSON blocks. Let me verify the syntax for each function call. Alright, that should cover it.
</think>

tool_call
{"name": "get_weather", "arguments": {"location": "Tokyo, Japan"}}
tool_call
{"name": "get_time", "arguments": {"location": "Tokyo, Japan"}}

Extracted Tool Call: [{'name': 'get_weather', 'arguments': {'location': 'Tokyo, Japan'}}, {'name': 'get_time', 'arguments': {'location': 'Tokyo, Japan'}}]


## Example 3: Force a Specific Function (`get_time`)

You can force the model to always call a specific function, regardless of the user's question.

In [6]:
response = client.chat.completions.create(
    model="Qwen/Qwen3-8B",
    messages=[
        {
            "role": "system",
            "content": (
                "You are a function calling assistant.\n"
                "You must always call the function get_time.\n"
                "Respond ONLY in the following format:\n"
                "<|im_start|>tool_call\n"
                '{"name": "get_time", "arguments": {"location": "<city, country>"}}\n'
                "<|im_end|>\n"
                "Do not answer the question directly or add any explanations. Only output the tool_call block."
            ),
        },
        {
            "role": "user",
            "content": "Tell me the time in New York."
        }
    ],
)
print("Raw Qwen Output:\n", response.choices[0].message.content)
tool_call = extract_tool_calls(response.choices[0].message.content)
print("\nExtracted Tool Call:", tool_call)

Raw Qwen Output:
 <think>
Okay, the user is asking for the time in New York. I need to figure out the correct location parameter for the get_time function. New York is a city in the United States, so the location should be "New York, USA". Let me make sure there's no other possible interpretation. The user didn't mention any other city or country, so it's safe to assume they want New York, USA. I'll call the get_time function with that location.
</think>

tool_call
{"name": "get_time", "arguments": {"location": "New York, USA"}}

Extracted Tool Call: [{'name': 'get_time', 'arguments': {'location': 'New York, USA'}}]


## Summary

- Qwen models use a special tool call format for function calling.
- You can simulate function calling by crafting your system prompt and parsing the output.
- This approach works with Qwen models on Featherless or any compatible endpoint.

---